### Rain in Australia - Predict next-day rain in Australia using logistic regression (Yes / no)
### please download the data from: https://rdrr.io/cran/rattle.data/man/weatherAUS.html and place it inside the data folder

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
# let us see what data available in our data directory
for direname, _ , filenames in os.walk("data"):
    for filename in filenames:
        print(os.path.join(direname, filename))

In [ ]:
# load data
data_path = "data/weatherAUS.csv"
df = pd.read_csv(data_path)
df.head()

### Exploratory data analysis

In [ ]:
# data shape
print(df.shape)
# column names
print(df.columns)

In [ ]:
# summary of dataset
print(df.info())

In [ ]:
# types of variables
# categrical variables
print(df["WindGustDir"].dtype)
categorical = [var for var in df.columns if df[var].dtype=='O']
print("There are {} categorical variables\n".format(len(categorical)))
print("The categorical variables are: ", categorical)

In [ ]:
# Drop RISK_MM variable
df.drop(['RISK_MM'], axis= 1, inplace= True)

In [ ]:
# check if the target variable has any na/null values
print(df.shape)
df = df[df['RainTomorrow'].notna()]
print(df.shape)

### Explore categorical variables

In [ ]:
# Explore categorical variables
df[categorical].head()

In [ ]:
# check missing variables
print(df[categorical].isnull().sum())
print("********\nPercentages of missing values\n********")
print(100 * df[categorical].isnull().sum() / df.shape[0])

In [ ]:
# check which of these variables has missing values?
cat_with_missing = [var for var in categorical if df[var].isnull().sum() > 0]
print(df[cat_with_missing].isnull().sum())

In [ ]:
# get the frequency counts of the categorical variables
for var in categorical:
    print(df[var].value_counts())

In [ ]:
# get the percentages in each of the categorical variables
for var in categorical:
    print(df[var].value_counts() / df.shape[0])

In [ ]:
# check cardinality (number of unique labels) of each catogrical variable
for var in categorical:
    us = df[var].unique()
    print(var, ' contains ', len(us), ' labels.')

In [ ]:
# parse the date variable in to year-month-day
df['Date'] = pd.to_datetime(df['Date']) # parse from strings to date time
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
# drop the date column
df.drop('Date', axis=1, inplace= True)
df.head()

In [ ]:
# a function to explore each of the categorical variables
def explore_categorical(df, var):
    # check if the variable has any missing values
    print('********** missing values **********')
    print(df[var].isnull().sum())
    print('********** Labels **********')
    # check unique lables in variable
    print(df[var].unique())
    print('********** frequency **********')
    # check frequency of each variable
    print(df[var].value_counts())

In [ ]:
explore_categorical(df, 'Location')

In [ ]:
explore_categorical(df, 'WindGustDir')

In [ ]:
explore_categorical(df, 'WindDir9am') 

In [ ]:
explore_categorical(df, 'WindDir3pm') 

In [ ]:
explore_categorical(df, 'RainToday') 

In [ ]:
explore_categorical(df, 'RainTomorrow') 

In [ ]:
# one-hot-encoding using pandas
pd.get_dummies(df['RainTomorrow'], drop_first=False, dummy_na=True)

### Explore Numerical Variables

In [ ]:
# find numerical variables
numericals = [var for var in df.columns if df[var].dtype != 'O']
print('There are {} numerical variables\n'.format(len(numericals)))
print('The numerical variables are :', numericals)
df[numericals].head()

In [ ]:
# Explore problems within numerical variables
# check missing values in numerical variables
df[numericals].isnull().sum()

In [ ]:
# view summary statistics in numerical variables
print(round(df[numericals].describe()), 2)

In [ ]:
# Let's draw boxplots to visualise outliers in these variables
plt.figure(figsize=(15, 10))
plt.subplot(2,2,1)
fig = df.boxplot(column= 'Rainfall')
fig.set_title('')
fig.set_ylabel('Rainfall')

plt.subplot(2,2,2)
fig = df.boxplot(column= 'Evaporation')
fig.set_title('')
fig.set_ylabel('Evaporation')

plt.subplot(2,2,3)
fig = df.boxplot(column= 'WindSpeed9am')
fig.set_title('')
fig.set_ylabel('WindSpeed9am')

plt.subplot(2,2,4)
fig = df.boxplot(column= 'WindSpeed3pm')
fig.set_title('')
fig.set_ylabel('WindSpeed3pm')

In [ ]:
# we can also use seaborn library to plot elegant ones
df_custom = df[['Rainfall', 'Evaporation', 'WindSpeed9am', 'WindSpeed3pm']]
import seaborn as sns
plt.figure(figsize=(15,10))
ax = sns.boxplot(data=df_custom, orient="h", palette="Set2")

In [ ]:
# plot histogram to check distribution
plt.figure(figsize=(15,10))


plt.subplot(2, 2, 1)
fig = df['Rainfall'].hist(bins=10)
fig.set_xlabel('Rainfall')
fig.set_ylabel('RainTomorrow')


plt.subplot(2, 2, 2)
fig = df.Evaporation.hist(bins=10)
fig.set_xlabel('Evaporation')
fig.set_ylabel('RainTomorrow')


plt.subplot(2, 2, 3)
fig = df.WindSpeed9am.hist(bins=10)
fig.set_xlabel('WindSpeed9am')
fig.set_ylabel('RainTomorrow')


plt.subplot(2, 2, 4)
fig = df.WindSpeed3pm.hist(bins=10)
fig.set_xlabel('WindSpeed3pm')
fig.set_ylabel('RainTomorrow')

In [ ]:
# Find aoutliers in these variables
def find_outliers(variable, factor= 3, print_summary=True):
    IQR = df[variable].quantile(0.75) - df[variable].quantile(0.25)
    Lower_boundary = df[variable].quantile(0.25) - (IQR * factor)
    Upper_boundary = df[variable].quantile(0.75) + (IQR * factor)
    
    outliers= []
    for index, val in enumerate(df[variable]):
        if val < Lower_boundary or val > Upper_boundary:
            outliers.append(index)
    
    
    if(print_summary):
        print('{variable} outliers are values < {lowerboundary} or > {upperboundary}'.format(variable= variable, lowerboundary=Lower_boundary, upperboundary=Upper_boundary))
    return Lower_boundary, Upper_boundary, outliers

In [ ]:
_,_,_ = find_outliers('Rainfall')

In [ ]:
_,_,_ = find_outliers('Evaporation')

In [ ]:
_,_,_ = find_outliers('WindSpeed9am')

In [ ]:
_,_,_ = find_outliers('WindSpeed3pm')

### Declare source and target variables

In [ ]:
X = df.drop(['RainTomorrow'], axis=1)
y = df['RainTomorrow']

### Split data into separate training and test set 

In [ ]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split

test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state=0)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

### Feature engineering

In [ ]:
X_train.dtypes

In [ ]:
# display categorical variables
categorical = [var for var in X_train.columns if X_train[var].dtypes == 'O']
categorical

In [ ]:
# display numerical variables
numericals = [var for var in X_train.columns if X_train[var].dtypes != 'O']
numericals

### Engineering missing values in numerical variables

In [ ]:
# display missing values
X_train[numericals].isnull().sum()

In [ ]:
# I could do the same for the text data
X_test[numericals].isnull().sum()

In [ ]:
# percentage of missing values in each variable
round(X_train[numericals].isnull().mean(), 2)

In [ ]:
# Impute the missing values with the median values -- median is robust with the outliers
for df_temp in [X_train, X_test]:
    for col in numericals:
        col_median = X_train[col].median() # get it only from training
        df_temp[col].fillna(col_median, inplace=True)

In [ ]:
# check again missing values in numerical variables in X_train
X_train[numericals].isnull().sum()

In [ ]:
# check missing values in numerical variables in X_test
X_test[numericals].isnull().sum()

### Engineering missing values in categorical variables

In [ ]:
round(X_train[categorical].isnull().mean(), 2)

In [ ]:
# impute missing categorical variables with most frequent value (i.e., mode)
for df_temp in [X_train, X_test]:
    for col in categorical:
        col_mode = X_train[col].mode()[0] # get it only from training
        df_temp[col].fillna(col_mode, inplace=True)

In [ ]:
# check missing values in categorical variables in X_train
X_train[categorical].isnull().sum()

In [ ]:
# check missing values in categorical variables in X_test
X_test[categorical].isnull().sum()

### Engineering outliers in numerical variables

In [ ]:
# Replace the outliers with some predefined the maximum value for each variable
def max_value(df_temp, variable, top):
    return np.where(df_temp[variable]>top, top, df_temp[variable])

cols_with_outliers = {'Rainfall': 3.2, 
                      'Evaporation': 21.8, 
                      'WindSpeed9am': 55, 
                      'WindSpeed3pm': 57
                     }
for df_temp in [X_train, X_test]:
    for col in cols_with_outliers:
        df_temp[col] = max_value(df_temp, col, cols_with_outliers[col])

In [ ]:
X_train.Rainfall.max(), X_test.Rainfall.max()

In [ ]:
X_train.Evaporation.max(), X_test.Evaporation.max()

In [ ]:
X_train.WindSpeed9am.max(), X_test.WindSpeed9am.max()

In [ ]:
X_train.WindSpeed3pm.max(), X_test.WindSpeed3pm.max()

In [ ]:
# we can also use seaborn library to plot elegant ones
df_custom = X_train[['Rainfall', 'Evaporation', 'WindSpeed9am', 'WindSpeed3pm']]
import seaborn as sns
plt.figure(figsize=(15,10))
ax = sns.boxplot(data=df_custom, orient="h", palette="Set2")

### Encoding the categorical variables

In [ ]:
X_train[categorical].head()

In [ ]:
# RainToday is binary, we can use the BinaryEncoder to create a one-hot encoder for it
import category_encoders as ce
encoder = ce.BinaryEncoder(cols=['RainToday'])

X_train = encoder.fit_transform(X_train)

X_test = encoder.transform(X_test)

In [ ]:
X_train.head()

In [ ]:
# Now let's create training set
X_train = pd.concat([X_train[numericals], X_train[['RainToday_0', 'RainToday_1']],
                     pd.get_dummies(X_train.Location), 
                     pd.get_dummies(X_train.WindGustDir),
                     pd.get_dummies(X_train.WindDir9am),
                     pd.get_dummies(X_train.WindDir3pm)], axis=1)

In [ ]:
X_train.head()

In [ ]:
# let's create test set
X_test = pd.concat([X_test[numericals], X_test[['RainToday_0', 'RainToday_1']],
                     pd.get_dummies(X_test.Location), 
                     pd.get_dummies(X_test.WindGustDir),
                     pd.get_dummies(X_test.WindDir9am),
                     pd.get_dummies(X_test.WindDir3pm)], axis=1)

In [ ]:
X_test.head()

### feature scaling or standardiztion

In [ ]:
# first keep the column name to get the DF back
cols = X_train.columns
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

# get the DFs back
X_train = pd.DataFrame(X_train, columns=[cols])
X_test = pd.DataFrame(X_test, columns=[cols])

X_train.head()

### Model training using logistic regression

In [ ]:
# train a logistic regression model on the training set
from sklearn.linear_model import LogisticRegression

# instantiate the model
logreg = LogisticRegression(solver='liblinear', random_state=0)


# fit the model
logreg.fit(X_train, y_train)

### Predict results

In [ ]:
y_pred_test = logreg.predict(X_test)

y_pred_test

### Check accuracy score 

In [ ]:
from sklearn.metrics import accuracy_score

print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred_test)))

### Confusion matrix 

In [ ]:
# Print the Confusion Matrix and slice it into four pieces

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred_test)

print('Confusion matrix\n\n', cm)

print('\nTrue Positives(TP) = ', cm[0,0])

print('\nTrue Negatives(TN) = ', cm[1,1])

print('\nFalse Positives(FP) = ', cm[0,1])

print('\nFalse Negatives(FN) = ', cm[1,0])

In [ ]:
# True Positives (Actual Positive:1 and Predict Positive:1) - 20892
# True Negatives (Actual Negative:0 and Predict Negative:0) - 3285
# False Positives (Actual Negative:0 but Predict Positive:1) - 1175 (Type I error)
# False Negatives (Actual Positive:1 but Predict Negative:0) - 3087 (Type II error)

In [ ]:
# visualize confusion matrix with seaborn heatmap

cm_matrix = pd.DataFrame(data=cm, columns=['Actual Positive:1', 'Actual Negative:0'], 
                                 index=['Predict Positive:1', 'Predict Negative:0'])

sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu')

In [ ]:
cm_normalised = cm.astype('float32') / cm.sum(axis=1)[:, np.newaxis]

# visualize confusion matrix with seaborn heatmap
cm_matrix = pd.DataFrame(data=cm_normalised, columns=['Actual Positive:1', 'Actual Negative:0'], 
                                 index=['Predict Positive:1', 'Predict Negative:0'])

sns.heatmap(cm_matrix, annot=True, fmt='.2f', cmap='YlGnBu')

### Other classification metrics

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_test))

### what are the other metrics that you could extract and interpret?

### ...